In [ ]:
server_ip    = '192.168.0.22' 
control_port = 54322
sensor_port  = 54324
flask_port   = 54323 
basic_auth_username = '31*hjbhv(41ok$'
basic_auth_password = '09&nb52#mcolz;'
video_fps=25

## Backend Sensor Logging Database

In [ ]:
import time, dateutil, json, queue, sqlite3, threading 

database_queue_in, database_queue_out = queue.Queue(), queue.Queue() 
def database_daemon(): # avoid sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread.
    EST = dateutil.tz.gettz('America/New York') # 'UTC'
    db = sqlite3.connect(":memory:")
    #try:
    #    os.remove("GPS.db")
    #    db = sqlite3.connect("GPS.db")
    #except:
    #    pass
    cur = db.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", ('tracking',))
    if not cur.fetchone():
        cur.execute("""CREATE TABLE tracking (gps_valid, temperature, x, y, z, voltage, latitude, longitude, altitude, course, speed, satellite, rover_localtime, readable_rover_localtime, server_localtime)""") 
    while True: 
        try:
            item = database_queue_in.get() 
            if item[0] == 'INSERT': 
                js = json.loads(item[1]) 
                if js['gps']: 
                    ts = dateutil.parser.isoparse(js['ts'])   
                    cur.execute("""INSERT INTO tracking (gps_valid, temperature,      x,      y,      z, voltage, latitude, longitude, altitude,   course,    speed, satellite, rover_localtime,                               readable_rover_localtime, server_localtime) 
                                                 VALUES (        ?,           ?,      ?,      ?,      ?,       ?,        ?,         ?,        ?,        ?,        ?,         ?,               ?,                                                      ?,                ?)""", 
                                                        (     True,     js['T'],js['x'],js['y'],js['z'],js['v'] ,js['lat'], js['lon'],js['lat'],js['cus'],js['spd'], js['sat'],  ts.timestamp(), ts.astimezone(EST).strftime("%Y-%m-%d %H:%M:%S (EST)"),      time.time()))  
                else:
                    cur.execute("""INSERT INTO tracking (gps_valid, temperature,      x,      y,      z, voltage,                                                                                                                                        server_localtime) 
                                                 VALUES (        ?,           ?,      ?,      ?,      ?,       ?,                                                                                                                                                       ?)""", 
                                                        (    False,     js['T'],js['x'],js['y'],js['z'],js['v'] ,                                                                                                                                             time.time()))   
            elif item[0] == 'newcomer':  #                    earliest gps positition - Home Spot
                cur.execute("""SELECT longitude, latitude, readable_rover_localtime, server_localtime FROM tracking 
                               WHERE server_localtime=(SELECT MIN(server_localtime) FROM tracking WHERE gps_valid==1)""") 
                database_queue_out.put(cur.fetchone()) 
            elif item[0] == 'trail':     #                    Trail   all valid record
                cur.execute("""SELECT longitude, latitude, readable_rover_localtime FROM tracking 
                               WHERE server_localtime>=? AND gps_valid==1""", (float(item[1]),))    
                database_queue_out.put(cur.fetchall())  
            elif item[0] == 'telemeter': #                    Trail   all valid record
                cur.execute("""SELECT temperature, x, y, z, voltage FROM tracking
                               WHERE server_localtime=(SELECT MAX(server_localtime) FROM tracking)""")    
                database_queue_out.put(cur.fetchone())
            elif item[0] == 'debug':     
                cur.execute(item[1]) 
                database_queue_out.put(cur.fetchall()) 
            database_queue_in.task_done()
            #if item[0]!='INSERT': print(item) 
        except Exception as e:
            print(e,'\t|\t\t\t\t|\t', item)
threading.Thread(target=database_daemon).start()  

def test_db():
    s = '{"T":26,"x":359.94,"y":0.44,"z":-0.56,"v":0.03,"lat":1.1,"lon":-2.3,"alt":37.70,"cus":5.21,"spd":0.87,"sat":7,"ts":"2021-01-17T15:47:41Z","gps":1}'
    database_queue_in.put(('INSERT', s))
    time.sleep(.1)
    s = '{"T":26,"x":359.94,"y":0.44,"z":-0.56,"v":0.03,"lat":1.2,"lon":-2.4,"alt":37.70,"cus":5.21,"spd":0.87,"sat":7,"ts":"2021-01-17T15:47:42Z","gps":0}'
    database_queue_in.put(('INSERT', s))
    time.sleep(.1)
    s = '{"T":26,"x":359.94,"y":0.44,"z":-0.56,"v":0.03,"lat":1.3,"lon":-2.5,"alt":37.70,"cus":5.21,"spd":0.87,"sat":7,"ts":"2021-01-17T15:47:45Z","gps":0}'
    database_queue_in.put(('INSERT', s))
    time.sleep(.1)
    s = '{"T":26,"x":359.94,"y":0.44,"z":-0.56,"v":0.03,"lat":1.4,"lon":-2.6,"alt":37.70,"cus":5.21,"spd":0.87,"sat":7,"ts":"2021-01-17T15:47:47Z","gps":1}'
    database_queue_in.put(('INSERT', s))
    time.sleep(.1)
    s = '{"T":26,"x":359.94,"y":0.14,"z":-0.56,"v":0.03,"lat":1.5,"lon":-2.7,"alt":37.70,"cus":5.21,"spd":0.87,"sat":7,"ts":"2021-01-17T15:47:49Z","gps":1}'
    database_queue_in.put(('INSERT', s)) 
    
    database_queue_in.put(('debug', "SELECT * FROM tracking;")) 
    print(database_queue_out.get())
    print('\n\n\n')
    
    database_queue_in.put(('telemeter', )) 
    temperature, x, y, z, voltage = database_queue_out.get() 
    print(temperature, x, y, z, voltage, '\n\n\n')
    
    database_queue_in.put(('newcomer', )) 
    longitude, latitude, rover_localtime, oldest_server_localtime = database_queue_out.get()  
    print(longitude, latitude, rover_localtime, oldest_server_localtime, '\n\n\n') 
    
    database_queue_in.put(('trail', oldest_server_localtime)) 
    print(database_queue_out.get()) 
    
    database_queue_in.put(('debug', "DELETE FROM tracking;")) 
#test_db()

## Sensor Link

## Control Link

## Image Link
pip install opencv-python 

In [ ]:
import cv2, PIL.Image, PIL.ImageDraw, PIL.ImageFont, io
class MJPEG():# image cache
    def __init__(self): 
        img = PIL.Image.new("RGB", (640, 480), 'white')  
        PIL.ImageDraw.Draw(img).text((75,200), "Image is not ready!", font=PIL.ImageFont.truetype("Pillow/Tests/fonts/DejaVuSans.ttf", 50), fill='black') 
        still_img = io.BytesIO() 
        img.save(still_img,format='JPEG')
        self.frame = still_img.getvalue()
        threading.Thread(target=self.updater).start()
         
    def updater(self):
        cap = cv2.VideoCapture("tcp://127.0.0.1:2222") 
        print('[Image link] Start capturing')
        while(cap.isOpened()):   
            ret, frame = cap.read() 
            if not ret:
                # print('continue')
                continue 
            ret, jpeg  = cv2.imencode('.jpg', frame)
            self.frame = jpeg.tobytes() 
            
mjpeg = MJPEG()

## Web Front
pip install flask flask_basicauth flask_cors 

In [ ]:
import flask, flask_basicauth, flask_cors 
with open('src/map.html', 'r') as fr: mapPage = fr.read()
app = flask.Flask(__name__)
flask_cors.CORS(app) 
app.config['BASIC_AUTH_USERNAME'] = basic_auth_username
app.config['BASIC_AUTH_PASSWORD'] = basic_auth_password
basic_auth = flask_basicauth.BasicAuth(app)
#log = logging.getLogger('werkzeug')
#log.setLevel(logging.ERROR) 
interval = 1/video_fps
@app.route("/")
#@basic_auth.required
def home(): 
    return flask.render_template_string(mapPage, 
        ip_port       = f'{server_ip}:{flask_port}', 
        username      =  basic_auth_username, 
        password      =  basic_auth_password)  
    
@app.route("/imageready", methods=['GET'])
#@basic_auth.required
def imageready(): 
    if mjpeg.frame:
        return flask.jsonify({"imageready":True}) 
    else:
        return flask.jsonify({"imageready":False}) 

@app.route("/stream.mjpg")
#@basic_auth.required
def stream():  
    def generator():      
        while True:     
            time.sleep(interval)  # threading.condition is too shitty according to my test. no condition no lag. 
            frame = mjpeg.frame  
            yield f'''--FRAME\r\nContent-Type: image/jpeg\r\nContent-Length: {len(frame)}\r\n\r\n'''.encode() 
            yield frame
    r = flask.Response(response=generator(), status=200)
    r.headers.extend({'Age':0, 'Content-Type':'multipart/x-mixed-replace; boundary=FRAME',
                      'Pragma':'no-cache', 'Cache-Control':'no-cache, private',}) 
    return r

def read_sql(put): 
    database_queue_in.join()
    database_queue_in.put(put)
    get = q_out.get()
    q_out.task_done()  
    return get 

@app.route("/telemeter", methods=['GET'])
#@basic_auth.required
def telemeter():
    _ = read_sql(("telemeter", ))
    if _: 
        temp, y, z, v = _  
        RESPONSE={'telemeterReady':True, 'temp':temp, 'y':y, 'z':z, 'v':v}
    else:
        RESPONSE={'telemeterReady':False} 
    return flask.jsonify(RESPONSE)
    
@app.route("/gps", methods=['GET'])
#@basic_auth.required
def gps():
    newestTimestampAtClientSide = flask.request.args.get('q')
    if newestTimestampAtClientSide == 'newcomer':
        _ = read_sql(("newcomer", ))
        if _: # There is valid GPS data
            lon, lat, localtime, newestTimestampAtServerSide = _  
            RESPONSE={ 
                "HomeSpot": [lat, lon], # lat lon in Google and Openstreetmap
                "newestTimestampAtServerSide": newestTimestampAtServerSide 
            }
        else:
            RESPONSE={"newestTimestampAtServerSide": "newcomer"}
    else: # newestTimestampAtClientSide is latest timestamp
        _ = read_sql(('trail', newestTimestampAtClientSide))
        if _: #
            lon, lat, localtime, newestTimestampAtServerSide, speed, x = _[-1] 
            if float(newestTimestampAtClientSide) == newestTimestampAtServerSide: # no new timestamp from database
                RESPONSE={"newestTimestampAtServerSide": newestTimestampAtClientSide}
            else: 
                RESPONSE={
                    "trailReady": True,
                    "CurrentSpot": [lat, lon], # lat lon in Google and Openstreetmap
                    "speed": speed,
                    "x": x,
                    "newestTimestampAtServerSide": newestTimestampAtServerSide, 
                    "trail": {
                        "type": "Feature",
                        "geometry": {
                            "type": "LineString",
                            "coordinates": [i[:2] for i in _]
                },},}
        else:
            RESPONSE={"trailReady": False}
    return flask.jsonify(RESPONSE)

@app.route('/axios.min.js', methods=['GET'])
def axios_js():
        return flask.send_file('src/axios.min.js', mimetype='application/javascript')
    
@app.route('/leaflet.rotatedMarker.js', methods=['GET'])
def rotatedMarker_js():
        return flask.send_file('src/leaflet.rotatedMarker.js', mimetype='application/javascript')
     
@app.route('/rainbow.svg', methods=['GET'])
def rainbow_svg():
        return '''<svg viewBox="0 0 640 480">
                      <polygon points="  0,0   80,0   80,480    0,480" style="fill:yellow;  " />  
                      <polygon points=" 80,0  160,0  160,480   80,480" style="fill:red;   " />  
                      <polygon points="160,0  240,0  240,480  160,480" style="fill:lime; " />  
                      <polygon points="240,0  320,0  320,480  240,480" style="fill:blue;  " />  
                      <polygon points="320,0  400,0  400,480  320,480" style="fill:hotpink;" />  
                      <polygon points="400,0  480,0  480,480  400,480" style="fill:orange;" />  
                      <polygon points="480,0  560,0  560,480  480,480" style="fill:magenta;" />  
                      <polygon points="560,0  640,0  640,480  560,480" style="fill:cyan;  " />  
                  </svg>'''
    
@app.route('/home.svg', methods=['GET'])
def home_svg():
        return '''<svg viewBox="0 0 20 32">
                      <path d="M 10.033061,0 C 4.4992758,0 -0.00100117,5.316 -0.00100117,12.001 c 0,7 5.01786697,14.161 8.67611897,19.194 0.01338,0.02 0.600371,0.805 1.3261692,0.805 0.0017,0 0.06104,0 0.06438,0 0.724962,0 1.312791,-0.785 1.326169,-0.805 3.659925,-5.033 8.676956,-12.193 8.676956,-19.194 C 20.068797,5.316 15.567684,0 10.033061,0 Z m 0.09783,29.883 c -0.01756,0.02 -0.06857,0.064 -0.112883,0.098 -0.0084,-0.027 -0.07024,-0.086 -0.107866,-0.133 C 6.8455738,25.631 1.6713428,18.514 1.6713428,12.001 1.6713428,6.487 5.4232458,2 10.033061,2 c 4.612325,0 8.363392,4.487 8.363392,10.002 0,6.512 -5.174231,13.629 -8.26556,17.881 z" style="fill:#ff0000;stroke-width:0.9144243" />
                      <path d="m 9.2992778,16.661416 c -1.830014,-0.280934 -3.419516,-1.649436 -3.996247,-3.440622 -0.230568,-0.716088 -0.282322,-1.759674 -0.123607,-2.492466 0.148539,-0.685809 0.613472,-1.6477058 1.036355,-2.1441059 0.673036,-0.7900409 1.603681,-1.3836639 2.594634,-1.6550184 0.595853,-0.1631634 1.8366892,-0.1659897 2.4406792,-0.00556 2.224842,0.5909569 3.728814,2.5383628 3.728814,4.8282233 0,1.921892 -1.081173,3.643515 -2.810451,4.475264 -0.839186,0.403632 -1.962582,0.573612 -2.8701772,0.434285 z" style="fill:#ff0000;fill-opacity:1;stroke-width:0.06779661" />
                  </svg>'''
    
@app.route('/rover.svg', methods=['GET'])
def arrow_svg():
        return '<svg viewBox="0 0 14 30"> <path d="M 0 30 L 7 0 L 14 30 C 7 23 7 23 0 30" style="fill:red;"></path></svg> '

def flask_in_bg():
    app.run(server_ip, flask_port)#, ssl_context=(server_cert_letsencrypt, server_key_letsencrypt)) 

print('https://guoxiaokang.net:54323/stream.mjpg')
print(f'http://{server_ip}:54323/stream.mjpg')
flask_in_bg()